In [3]:
%%configure -f
{"conf": {"spark.jars": "/jar/simpleHTM.jar,/jar/jhealpix.jar"}}

In [9]:
import simple.HTMindex
import healpix.jhu.Healpix
import org.apache.spark.sql.{SparkSession, SaveMode, Row, DataFrame}
import org.apache.spark.sql.functions.udf
import java.util.Calendar



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import simple.HTMindex
import healpix.jhu.Healpix
import org.apache.spark.sql.{SparkSession, SaveMode, Row, DataFrame}
import org.apache.spark.sql.functions.udf
import java.util.Calendar


In [5]:
object HTMUtils extends Serializable {
    var htmindex= new HTMindex() with Serializable
    val htmid: (Double, Double) => Long = htmindex.lookupId(_,_)

    val htmidUDF=udf(htmid)
    
}

object HEALPixUtils extends Serializable {
    var hp = new Healpix() with Serializable
    val healpixid: (Double, Double) => Long = hp.ang2pix(_,_)
    val healpixidUDF=udf(healpixid)
    
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined object HTMUtils
defined object HEALPixUtils


read object table parquet files into dataframe (objDF)

create new dataframe (newDF) with calculated htmid

In [7]:
val datafile = "/user/hive/warehouse/object_test_parquet"
val objDF = spark.read.parquet(datafile)

val newDF = objDF.withColumn("htmid",HTMUtils.htmidUDF(objDF("ra"),objDF("decl"))).withColumn("healpixid", HEALPixUtils.healpixidUDF(objDF("ra"), objDF("decl")))

//val newDF = objDF.withColumn("htmid",HTMUtils.htmidUDF(objDF("ra"),objDF("decl")))
//newDF.printSchema()
//newDF.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

datafile: String = /user/hive/warehouse/object_test_parquet
objDF: org.apache.spark.sql.DataFrame = [deepSourceId: bigint, ra: double ... 234 more fields]
newDF: org.apache.spark.sql.DataFrame = [deepSourceId: bigint, ra: double ... 236 more fields]


write newDF to external table in SqlDataPool using Spark-SQL connector


In [11]:

println("Use MSSQL connector to write to master SQL instance ")

val servername = "jdbc:sqlserver://master-0.master-svc"
val dbname = "LSST"
var url = servername + ";" + "databaseName=" + dbname + ";"

val user = "sa"
val password = "fooRiuzg54" //Please specify password here

val datapool_table = "ObjectDP_htmid_healpix_test"

val datasource_name = "SqlDataPool"

val batchsize = 1000000

val start = Calendar.getInstance().getTime()

try {
  newDF.write 
    .format("com.microsoft.sqlserver.jdbc.spark") 
    .mode("append") 
    .option("url", url) 
    .option("dbtable", datapool_table) 
    .option("user", user) 
    .option("password", password) 
    .option("dataPoolDataSource",datasource_name)
    .option("batchsize",batchsize)
    .save()
} catch {
    case e: Throwable => println("MSSQL Connector write failed: " + e)
}
val end = Calendar.getInstance().getTime()
println(start)
println(end)
//var duration = end - start
//println("duration:" + duration)
print("MSSQL Connector write(append) to data pool external table succeeded")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use MSSQL connector to write to master SQL instance 
servername: String = jdbc:sqlserver://master-0.master-svc
dbname: String = LSST
url: String = jdbc:sqlserver://master-0.master-svc;databaseName=LSST;
user: String = sa
password: String = fooRiuzg54
datapool_table: String = ObjectDP_htmid_healpix_test
datasource_name: String = SqlDataPool
batchsize: Int = 1000000
start: java.util.Date = Tue Jan 21 20:28:59 UTC 2020
end: java.util.Date = Tue Jan 21 20:29:49 UTC 2020
Tue Jan 21 20:28:59 UTC 2020
Tue Jan 21 20:29:49 UTC 2020
MSSQL Connector write(append) to data pool external table succeeded